In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pygmo import *

In [5]:
# This class was modeled from the pygmo Documentation. 
# This class is required for me to be able to run the PSO algorithm from 'pygmo'
# The 'best_fit' that you see here is declared inside the 'pso_optimizer' function
class F3:
    def __init__(self, given_bias, given_bounds, given_dimensions):
        
        self.given_bias = given_bias
        self.given_bounds = given_bounds 
        self.given_dimensions = given_dimensions

    def fitness(self, x):
        global best_fit
        
        Function = 0
        
        z_func = np.sum(np.array(x) - np.array(f3_shift_values[:self.given_dimensions])) + np.ones(self.given_dimensions)
        
        for i in range(self.given_dimensions-1):
            Function += 100 * ((z_func[i]**2 - z_func[i+1])**2) + (z_func[i] - 1)**2
        
        rosenbrock_function = Function + self.given_bias
        best_fit.append(rosenbrock_function)
        
        return [rosenbrock_function]

    def get_bounds(self):
        return ([self.given_bounds[0]] * self.given_dimensions, [self.given_bounds[1]] * self.given_dimensions)


In [6]:
# Partical Swarm Optimization Function is Defined Below 

def pso_optimizer(given_bias, given_bounds, given_dimensions):
    
####Parameters Intialization
    
    # Population size list
    pop_list = [50, 100, 150]
    
    # omega in pygmo
    inertia_factor_list = [0.3,0.5,0.7,0.9]
    
    # eta1: Congnitive component in pygo
    self_confidence_list = [1,2,3]
    
    # eta2: Socialcomponent in pygmo
    swarm_confidence_list = [1,2,3]
    
    #vcoeff : Maximum Velocity Coefficient with bound width [0,1]
    vcoeff_list = [0.3,0.5,0.7,0.9]
    
    
    
    global best_fit
    current_optimal_fitness_value = np.inf
    # Best fitness results are all saved in these lists
    optimal_parameter_fitness = []
    optimal_parameter_result = []
    optimal_parameter_inertia_factor = []
    optimal_parameter_self_confidence = []
    optimal_parameter_swarm_confidence = []
    optimal_parameter_vcoeff = []
    optimal_parameter_pop = []
    optimal_parameter_best_fit_curve = [] 
    
    
    # Problem Definition in Pygmo
    p = problem(F3(given_bias,given_bounds,given_dimensions))
    
    # The solving part 
    for inertia_factor in inertia_factor_list:
        for self_confidence in self_confidence_list:
            for swarm_confidence in swarm_confidence_list:
                for vcoeff in vcoeff_list:
                    for pop in pop_list:
                        best_fit = []
                        
                        #PSO and Population object is defined here 
                        Population = population(p,pop)
                        pso_algo = algorithm(pso(gen=1200, omega = inertia_factor, eta1= self_confidence, eta2 = swarm_confidence,max_vel=vcoeff,seed=4)) 
                                                            # I wanted to use more than 1000 but due to time contraints i can't
                                                           # takes too much time to run so the results i get with 1200 should do
                        
                        final_result = pso_algo.evolve(Population)
                        
                        # Results are now appended in all of the empty optimal parameter lists that were defined in the above
                        if final_result.champion_f < current_optimal_fitness_value:
                            current_optimal_fitness_value = final_result.champion_f
                            optimal_parameter_fitness.append(final_result.champion_f)
                            optimal_parameter_result.append(final_result.champion_x)
                            optimal_parameter_inertia_factor.append(inertia_factor)
                            optimal_parameter_self_confidence.append(self_confidence)
                            optimal_parameter_swarm_confidence.append(swarm_confidence)
                            optimal_parameter_vcoeff.append(vcoeff)
                            optimal_parameter_pop = pop
                            optimal_parameter_best_fit_curve = best_fit
                            
    print("The Optimal Parameters are printed here below")
    print("Optimal Interia Factor: ", optimal_parameter_inertia_factor )
    print("Optimal Self Confidence: ", optimal_parameter_self_confidence)
    print("Optimal Swarm Confidence: ", optimal_parameter_swarm_confidence)
    print("Optimal Maximum Partical Velocity: ", optimal_parameter_vcoeff)
    print("Optimal Population Size: ", optimal_parameter_pop)
    print("The Best fitness: ", optimal_parameter_fitness)
    print("The optimal Result: ", optimal_parameter_result)
    
    # For generation 1200 that was specified the min of each swarm
    curve = np.array([np.min(np.array(optimal_parameter_best_fit_curve[i:i+optimal_parameter_pop])) for i in range(0,len(optimal_parameter_best_fit_curve), optimal_parameter_pop)])
    
    plt.plot(curve)
    
    if given_dimensions == 50:
        plt.savefig("curve_dimension_50_Rosebrock_pso.png")
        result_df = pd.DataFrame({'solution':optimal_parameter_result}) 
        result_df.to_csv("Result_pso_50_Rosenbrock.csv")
    
    else:
        plt.savefig("curve_dimension_500_Rosenbrock_pso.png")
        # Saving solution to csv
        result_df1 = pd.DataFrame({'solution':optimal_parameter_result}) 
        result_df1.to_csv("Result_pso_500_Rosenbrock.csv")                                     
                
    

In [ ]:
if __name__=="__main__":
    
    f3_shift_values = []
    
    with open('F3_shift_values.txt','r') as f:
        for line in f:
            f3_shift_values.append(float(line[:-1]))
              
    # Parameters for F3

    given_bias = -450
    given_dimensions = (50,500)
    given_bounds = (-100,100)
              
    best_fit = []
              
    print("Dimension 50 - PSO_Rosenbrock\n")
              
    pso_optimizer(given_bias, given_bounds, given_dimensions[0])
              
    print("Dimension 500 - PSO_Rosenbrock\n")
              
    pso_optimizer(given_bias, given_bounds, given_dimensions[1])

Dimension 50 - PSO_Rosenbrock

The Optimal Parameters are printed here below
Optimal Interia Factor:  [0.3]
Optimal Self Confidence:  [1]
Optimal Swarm Confidence:  [3]
Optimal Maximum Partical Velocity:  [0.5]
Optimal Population Size:  100
The Best fitness:  [array([-450.])]
The optimal Result:  [array([ 48.52852597, -84.20619294, -51.97905385,  24.62453276,
        32.67730486,  44.6181206 ,  63.96422288, -30.88243988,
        -4.49019724,  96.59300985, -61.82512142,  -6.02415454,
       -18.84530152,  15.2504089 ,  67.79493912,   4.16010907,
        20.50239863,  72.95265965,  -4.92949069, -27.61718346,
       -64.87405785, -76.55939468,  70.63523875,  22.86752661,
       -99.98445524, -45.06177164,  39.97123996,  45.15322797,
       -15.43699868,  12.78327767,  29.63613938,  13.96470326,
       -75.51856732,  12.61839986, -35.72322141, -62.25766048,
       -21.11625831,  91.91006605, -60.23640582, -45.92845304,
        31.80543815,  20.09700632, -12.8722923 ,  39.83731146,
        